In [ ]:
import numpy as np
import pandas as pd
import itertools
from geopy import Point
from geopy.distance import distance


In [ ]:
cities = pd.read_csv('./data/esri-usa-major-cities.csv')
cols_of_interest = [
    'X',
    'Y',
    'NAME',
    'CLASS',
    'PLACEFIPS',
    'CAPITAL',
    'POP_CLASS',
    'POPULATION',
]
cities = cities[cols_of_interest]
cities = cities[cities['CLASS'] == 'city']
big_cities = cities[cities.POPULATION > 500000]
big_cities = big_cities.reset_index().drop(columns='index')

big_cities['latlong'] = list(zip(big_cities['X'], big_cities['Y']))
big_cities = big_cities.drop(columns=['X', 'Y'])
big_cities

In [ ]:
big_cities_cross = big_cities.merge(big_cities, how='cross', suffixes=('_dep', '_arr'))

# idx = big_cities_cross['NAME_dep'] != big_cities_cross['NAME_arr']
# big_cities_cross = big_cities_cross[idx]
# todo remove (x,y) if (y,x) present in dataframe
# comb = itertools.combinations(big_cities['NAME'], r=2)
big_cities_cross = big_cities_cross.reset_index().drop(columns='index')

# assume annual passenger volume (in each direction) is 5% of the combined population of the two cities
passenger_vol = 0.05 * (big_cities_cross['POPULATION_dep'] + big_cities_cross['POPULATION_arr'])
big_cities_cross['passenger_vol'] = passenger_vol

big_cities_cross.apply(lambda row: distance(row['latlong_dep'], row['latlong_arr']), axis='columns')